https://www.youtube.com/watch?v=W7ktPe1HfZs&list=PLPTV0NXA_ZSiOpKKlHCyOq9lnp-dLvlms&index=22

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(42)

Note: wget does not work in the VS Code terminal, so we use curl instead.

In [ ]:
!wget https://raw.githubusercontent.com/Avisoori1x/makeMoE/main/input.txt

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
!curl -O https://raw.githubusercontent.com/Avisoori1x/makeMoE/main/input.txt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 71 1089k   71  780k    0     0   547k      0  0:00:01  0:00:01 --:--:--  548k
100 1089k  100 1089k    0     0   741k      0  0:00:01  0:00:01 --:--:--  742k


In [ ]:
class Expert(nn.Module):
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed)
            nn.Dropout(dropout),  # Add dropout for regularization
        )

    def forward(self, x):
        return self.net(x)
        

In [ ]:
num_experts = 3
top_k = 2
n_embed = 8

mh_output = torch.randn(2, 4, n_embed)

# routing matrix
topkgate_linear = nn.Linear(n_embed, num_experts)

logits = topkgate_linear(mh_output)

print("Logits shape:", logits.shape)
print("Logits:", logits)

Logits shape: torch.Size([2, 4, 3])
Logits: tensor([[[-1.5546, -0.2941,  1.3507],
         [-0.2603,  0.2799,  0.7907],
         [-0.9108,  0.3964,  0.4110],
         [-0.2563, -0.4846, -0.7813]],

        [[ 0.4986,  0.4953, -0.3828],
         [ 0.0560,  0.7386, -0.6886],
         [-0.5285, -0.3301,  1.5522],
         [-0.3325,  0.4946,  0.2142]]], grad_fn=<ViewBackward0>)


In [9]:
top_k_logits, top_k_indices = logits.topk(top_k, dim=-1)

print("Top-k logits:", top_k_logits)
print("Top-k indices:", top_k_indices)

Top-k logits: tensor([[[ 1.3507, -0.2941],
         [ 0.7907,  0.2799],
         [ 0.4110,  0.3964],
         [-0.2563, -0.4846]],

        [[ 0.4986,  0.4953],
         [ 0.7386,  0.0560],
         [ 1.5522, -0.3301],
         [ 0.4946,  0.2142]]], grad_fn=<TopkBackward0>)
Top-k indices: tensor([[[2, 1],
         [2, 1],
         [2, 1],
         [0, 1]],

        [[0, 1],
         [1, 0],
         [2, 1],
         [1, 2]]])


In [12]:
zeros = torch.full_like(logits, fill_value=float("-inf"))

print("Zeros:", zeros)

sparse_logits = zeros.scatter(
    dim=-1,
    index=top_k_indices,
    src=top_k_logits
)

print("Sparse logits:", sparse_logits)

Zeros: tensor([[[-inf, -inf, -inf],
         [-inf, -inf, -inf],
         [-inf, -inf, -inf],
         [-inf, -inf, -inf]],

        [[-inf, -inf, -inf],
         [-inf, -inf, -inf],
         [-inf, -inf, -inf],
         [-inf, -inf, -inf]]])
Sparse logits: tensor([[[   -inf, -0.2941,  1.3507],
         [   -inf,  0.2799,  0.7907],
         [   -inf,  0.3964,  0.4110],
         [-0.2563, -0.4846,    -inf]],

        [[ 0.4986,  0.4953,    -inf],
         [ 0.0560,  0.7386,    -inf],
         [   -inf, -0.3301,  1.5522],
         [   -inf,  0.4946,  0.2142]]], grad_fn=<ScatterBackward0>)


In [13]:
gating_output = F.softmax(sparse_logits, dim=-1)

print("Gating output:", gating_output)

Gating output: tensor([[[0.0000, 0.1618, 0.8382],
         [0.0000, 0.3750, 0.6250],
         [0.0000, 0.4963, 0.5037],
         [0.5568, 0.4432, 0.0000]],

        [[0.5008, 0.4992, 0.0000],
         [0.3357, 0.6643, 0.0000],
         [0.0000, 0.1321, 0.8679],
         [0.0000, 0.5696, 0.4304]]], grad_fn=<SoftmaxBackward0>)


In [15]:
class TopkRouter(nn.Module):
    def __init__(self, m_embed, num_experts, top_k):
        super().__init__()        
        self.top_k = top_k
        self.linear = nn.Linear(m_embed, num_experts)
    
    def forward(self, mh_output):
        logits = self.linear(mh_output)
        top_k_logits, top_k_indices = logits.topk(self.top_k, dim=-1)
        zeros = torch.full_like(logits, fill_value=float("-inf"))
        sparse_logits = zeros.scatter(
            dim=-1,
            index=top_k_indices,
            src=top_k_logits
        )
        gating_output = F.softmax(sparse_logits, dim=-1)
        return gating_output, top_k_indices

In [16]:
# test the TopkRouter

num_experts = 3
top_k = 2
m_embed = 8

mh_output = torch.randn(1, 4, m_embed)

router = TopkRouter(m_embed, num_experts, top_k)

gating_output, top_k_indices = router(mh_output)

print("Gating output shape:", gating_output.shape)
print("Gating output:", gating_output)
print("Top-k indices shape:", top_k_indices.shape)
print("Top-k indices:", top_k_indices)

Gating output shape: torch.Size([1, 4, 3])
Gating output: tensor([[[0.0000, 0.5124, 0.4876],
         [0.5763, 0.0000, 0.4237],
         [0.6128, 0.0000, 0.3872],
         [0.4348, 0.0000, 0.5652]]], grad_fn=<SoftmaxBackward0>)
Top-k indices shape: torch.Size([1, 4, 2])
Top-k indices: tensor([[[1, 2],
         [0, 2],
         [0, 2],
         [2, 0]]])


In [17]:
class NoisyTopkRouter(nn.Module):
    def __init__(self, m_embed, num_experts, top_k):
        super().__init__()
        self.top_k = top_k
        self.linear = nn.Linear(m_embed, num_experts)
        self.noise_linear = nn.Linear(m_embed, num_experts)

    def forward(self, mh_output):
        logits = self.linear(mh_output)

        noise_logits = self.noise_linear(mh_output)

        noise = torch.randn_like(logits) * F.softplus(noise_logits)
        noisy_logits = logits + noise

        top_k_logits, top_k_indices = noisy_logits.topk(self.top_k, dim=-1)
        zeros = torch.full_like(logits, fill_value=float("-inf"))
        sparse_logits = zeros.scatter(
            dim=-1,
            index=top_k_indices,
            src=top_k_logits
        )
        gating_output = F.softmax(sparse_logits, dim=-1)
        return gating_output, top_k_indices




In [18]:
# test the NoisyTopkRouter

num_experts = 3
top_k = 2
m_embed = 8

mh_output = torch.randn(1, 4, m_embed)

router = NoisyTopkRouter(m_embed, num_experts, top_k)

gating_output, top_k_indices = router(mh_output)

print("Gating output shape:", gating_output.shape)
print("Gating output:", gating_output)
print("Top-k indices shape:", top_k_indices.shape)
print("Top-k indices:", top_k_indices)

Gating output shape: torch.Size([1, 4, 3])
Gating output: tensor([[[0.9068, 0.0000, 0.0932],
         [0.5094, 0.4906, 0.0000],
         [0.0000, 0.5448, 0.4552],
         [0.5408, 0.4592, 0.0000]]], grad_fn=<SoftmaxBackward0>)
Top-k indices shape: torch.Size([1, 4, 2])
Top-k indices: tensor([[[0, 2],
         [0, 1],
         [1, 2],
         [0, 1]]])
